Q:怎样的问题适合用动态规划求解？

A:存在子问题，并且可以对子问题进行优化

第一步：确定动态规划状态

第二步：写出一个好的状态转移方程

第三步：考虑初始条件

第四步：考虑输出状态

第五步：考虑对时间，空间复杂度的优化（Bonus）

### 编辑距离

关键是将插入、删除和替换三个操作与子问题匹配。将当前状态dp[i][j]与过去的状态联系起来，例如dp[i-1][j-1] dp[i][j-1] dp[i-1][j]甚至可能相隔两个时刻。
对于插入操作，即向word1 or word2里添加某个字母，使得两者匹配，等同于将word1 or word2中的那个字母删掉。例如had与head，向had中插入e等同于删掉head中的e，那么had[1] 与 head[1] （1代表索引h的位置0，e的位置是1）的比较就可以转换为had[1]和head[0]的比较。
对于替换操作，将当前word1[i]替换为与word2[j]相同的词，那就回归到word[i] == word2[j]的处理方式。

Q:这种处理方式有没有贪心的成分在，只需要对局部状态进行考虑？

A:不是局部的，在初始化dp方程时就已经将整个字符串考虑在内了。第一行与第一列分别是将空字符串转换到word1与word2所需的步骤。子问题是将word1的前一个、前两个、前三个……与word2比较

In [ ]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        m = len(word1)
        n = len(word2)

        dp = [[0]*(n+1) for i in range(m+1)]

        for i in range(m+1):
            dp[i][0] = i
        
        for j in range(n+1):
            dp[0][j] = j

        for i in range(1,m+1):
            for j in range(1,n+1):
                if word1[i-1] == word2[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    dp[i][j] = min(dp[i-1][j-1],dp[i-1][j],dp[i][j-1]) + 1
        return dp[-1][-1]

### 最长回文子串

暴力方法：取出所有str[i:j]的子序列情况，放入回文子串的判断中，时间复杂度是n3
将str进行转置，str[::-1]，找到与str的最长公共子序列

考虑子问题，比如对于acabadefffe这个字符串，从初始位置开始考虑即只有一个a，其最长回文子串就是它本身，记录目前最长回文子串的长度为1，加入一个字符c，那么多了一个考虑情况即ac是否回文，可以知道ac不是，最长回文子串的长度仍然是1，加入一个a，这时多了两个考虑情况，即aca与ca是否是回文，可知前者是，更新最长回文子串的长度为3，然后加入b，情况acab，然后加入a，考虑acaba以及caba，以此类推

对于acaeffe，根据当前的终点位置i和maxlen来对子区间进行考虑。

(1,1)对应考虑的区间就是[0:2]
(2,1)—— [0:3]是回文，maxlen+2 = 3
(3,3) [0:4]
(4,3) [0:5] [1:5]
(5,3) [1:6] [2:6] 这时不对[0:6]进行考虑，因为如果[0:6]是回文，那么[1:5]必然是回文，而[1:5]已经考虑过了。即新加入一个字符，最长回文子串的长度只可能加1或者加2
(6,3) [2:7] [3:7] 是回文maxlen+1 =4


In [ ]:
#转自https://leetcode.com/problems/longest-palindromic-substring/discuss/2925/Python-O(n2)-method-with-some-optimization-88ms.

class Solution:
    def longestPalindrome(self, s: str) -> str:
        if len(s)==0:
            return ''
        maxLen=1
        start=0
        for i in range(len(s)):
            if i-maxLen >=1 and s[i-maxLen-1:i+1]==s[i-maxLen-1:i+1][::-1]:
                                 #新加入一个字符，最长回文子串增加2个
                start=i-maxLen-1
                maxLen+=2
                continue       

            if i-maxLen >=0 and s[i-maxLen:i+1]==s[i-maxLen:i+1][::-1]:
                                #新加入一个字符，最长回文子串增加1个
                start=i-maxLen
                maxLen+=1
        return s[start:start+maxLen]
    
##使用动归
class Solution:
    def longestPalindrome(self, s: str) -> str:
        length = len(s)
        if length < 2:
            return s 
        dp = [[False]*length for i in range(length)]
        maxlen = 1
        start = 0
        for j in range(1,length):
            for i in range(j):
                if s[i] == s[j]:
                    if j - i < 3:
                        dp[i][j] = True
                    else:
                        dp[i][j] = dp[i+1][j-1]
                if dp[i][j]:
                    cur_len = j - i + 1
                    if cur_len > maxlen:
                        maxlen = cur_len 
                        start = i
        return s[start:start+maxlen]

In [ ]:
#中心扩散法，转自https://leetcode.com/problems/longest-palindromic-substring/discuss/2954/Python-easy-to-understand-solution-with-comments-(from-middle-to-two-ends).

class Solution:
    def longestPalindrome(self, s: str) -> str:
        res = ""
        for i in range(len(s)):
            # odd case, like "aba"
            tmp = self.helper(s, i, i)
            if len(tmp) > len(res):
                res = tmp
            # even case, like "abba"
            tmp = self.helper(s, i, i+1)
            if len(tmp) > len(res):
                res = tmp
        return res


    def helper(self, s, l, r):
        while l >= 0 and r < len(s) and s[l] == s[r]:
            l -= 1; r += 1
        return s[l+1:r]

### 打家劫舍

每个房间只有偷与不偷两个选项，而当前房间是否要偷，与前面的房间，以及前面的前面的房间有关

In [ ]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        if not nums:
            return 0
        if len(nums) == 1:
            return nums[0]
        dp = [0]*(len(nums))
        dp[0] = nums[0]
        dp[1] = max(nums[0],nums[1])
        for i in range(2,len(nums)):
            dp[i] = max(dp[i-2]+nums[i],dp[i-1])
        return dp[-1]

### 打家劫舍二

第一家和最后一家不能同时入选，那就分别考虑，取最大值

In [ ]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        def helper(nums):
            dp = [0] * (len(nums))
            dp[0] = nums[0]
            dp[1] = max(nums[0],nums[1])
            for i in range(2,len(nums)):
                dp[i] = max(dp[i-1],dp[i-2] + nums[i])
            return dp[-1]
        if not nums:
                return 0
        if len(nums) == 1:
            return nums[0]
        if len(nums) == 2:
            return max(nums)
        return max(helper(nums[1:]),helper(nums[:-1]